# Boids Simulation 2.0:
- The first simulation I wrote worked however it suffered from computational complexity O(n^2) for the distance calculations
- This code uses a different programming paradigm so that it can take advantage of certain computational advantages such as the parallelism of the GPU, Multiprocessing on multiple CPU cores, more efficient algorithms such as the Barnes Hut and more useful datastructures such as the KDTree. 
- 8/20/19 2:48pm It has finally dawned on my why strictly typed languages might be so useful for large projects ... While they certianly have their disadvantages pertaining to syntax and volume of code ... it is more clear what is being done... orginazation.... 
- Code by Michael Sherif naguib 7/22/19

### Imports

In [1]:
#imports (may take some time...)
from __future__ import print_function#A requirement of ipywidgets
from ipywidgets import interact, interactive, fixed, interact_manual# A python library for GUI interaction
from scipy.spatial import KDTree as KDTree# A library for constructing and querying a KDTree
import ipywidgets as widgets# A python library for GUI interaction
from numba import cuda, jit# An optimization library that can speed up code and even run on GPU
from tkinter import *# Python's library for creating GUI
import math# Python's library for math functions etc,,,
import random# Python's library for random numbers
import tqdm# A progress logger bar library for iteration etc...
import pptk# Static 3d point cloud visualization lib for large 10mil+ datapoints
import time# Python Library for time related things
import open3d# A library for LIDAR point cloud visualization and so much more 1k points
import numpy as np# a library for scientific computing
#import pyopencl as cl # a library for running computations on the GPU 

### Utilities & Etc Needed for Settings

In [2]:
phi = (1 + math.sqrt(5))/2#Prespecify a useful constant: The Golden Ratio
def softmax(inputList,b=1):#Calculate the softmax values for a list returning a new list
    #inputList: a list of numbers to calculate the corresponding softmax values for
    #b=1: a default value saying to use e as the base of exponentiation ... see this wikipedia article for info
    #     https://en.wikipedia.org/wiki/Softmax_function
    assert( not(len(inputList)==0))
    expSum = sum([math.exp(inputList[i]) for i in range(0,len(inputList))])#Sum e^x_i
    #note! could cache the exp calculations for e^x_i ... might save computationally especially if x_i is 'large'
    return [math.exp(inputList[i])/expSum for i in range(0,len(inputList))]# x_i --> e^x_i/(sum) 
def relativeWeight(inputList):
    s = sum(inputList)
    if not s ==0:
        return [i/s for i in inputList ]
    else:
        return copy.deepcopy(inputList)

### Settings

In [3]:
#================ SETTINGS
#=== Boid Quantity
boidQuantity = 100#int(math.pow(10,2))#                                      Quantity of boids in simulation
#=== Screen Display
height = 400 #                                                           Screen height 
dimensions = 3#                                                          Dimensions ex. 2D x,y 3D x,y,z (inclusive..)
posMax = np.array([height,int(height*phi),int(height*phi)])#             World bounds for Positions x,y,z max 
posMin = np.zeros(3)#                                                    x,y,z min ... use golden ratio for nice view
assert(posMin.size == posMax.size == dimensions)#         (err check)
minFrameTime = 0#                                                        Minimum time allowed between sucessive frames ( in seconds ... can do decimal)
#=== Kinematics 
maxForceMag = 2*math.pow(math.pi,2)#                                     Maximum magnitude for force update
maxAccelMag = math.pow(math.pi,2)#                                       Maximum magnitude for acceleration update
maxVelMag = math.pi#                                                     Maximum magnitude for velocity update
#=== Boid Properties
mass = np.ones(boidQuantity)#np.random.uniform(low=0.0001,high=2.0,size=boidQuantity)#         sets the mass of the boid
active = np.random.randint(0,high=2,size=boidQuantity,dtype=bool)#            an easy way to dynamically set a boid as active in the simulation
geneCountPer = 5#                                                        specifies allocation for the gene...
genomes = np.random.rand(boidQuantity,geneCountPer)#                     so we can run a genetic algorithm
boidScale = 1#                                                           the size of the dot to draw representing it
#=== Rule Distances
ruleCount = 3
maxDist = (1/2)*(math.sqrt(sum([math.pow(posMax[i]-posMin[i],2) for i in range(dimensions)])))# max dist = world bound max /2 because it wraps
minDist = 1
ruleDistances = np.random.uniform(low=minDist,high=maxDist,size=(boidQuantity,ruleCount))
#=== Screen Settings
backgroundColor = 'white'
boidColor = 'black'
boidColorRGBList = [0,0,0]
#=== Rule Settings
''' 
    We want the forces to interact in a relative manner ==> i.e one force is stronger than the other to a certain degree
    Therfore we can use the softmax function to convert our forces to a distribution
    Returns a new list containing the results of the softmax function on each... b sets for different bases default e=> b=1
'''
ruleList = []# a list holding the customized rule functions... (see section for initing these...)
distance_scale = 50 # set the influence zone... #order: Cohere,aligm,seperate
ruleDistances= [distance_scale*normDist for normDist in softmax([1,0.75,0.5])]#make the distances relative scaled by some factor
ruleWeights = relativeWeight([0.5,0.2,0.6])#Make the weight relative...




In [4]:
#TODO 
 '''
#=== Mode Settings
Record = False
availableModes = ["live","save","load"]#    select from this list
selectedMode= 0 #                         the index of the selected mode
#=== Mode specific settings (only used if the mode is used ...)
#Live (-- NO ADDITIONAL SETTINGS -- )
#Save
displayWhileComputing = True #    will show the simulation while it is being computed 
#NOTE!!!!! time delay might not be needed if you are not viewing! ==> set to 0 to get speedup
sFileName = "simulation.dat" # the filename to save to
#Load
lFileName = "simulation.dat" #the filename to read from

#=== GPU SETTINGS
# datatypes must be explicity set however you may wish to use a different type for the Real Numer values for 
#position acceleration etc... if it  would mean too much memory is needed ... 
#this will be entirely dependant on your system and the ammount of shared memory it has ... 
'''

IndentationError: unexpected indent (<ipython-input-4-9aee6fb5192e>, line 2)

### Initilization & Setup

In [5]:
#=== Init Kinematics Rand
pos = []
#distanceMatrix = np.zeros(boidQuantity,boidQuantity)#                    init a matrix of 0's for positions...  (NOT NEEDED SINCE USING KD TREE...)                  
vel = [np.random.rand(dimensions) for _ in range(boidQuantity)]#        init random vals on 0,1
accel = [np.random.rand(dimensions) for _ in range(boidQuantity)]#      init random vals on 0,1
#==== Position
for i in range(dimensions):#                                             init random positions in bound
    pos.append(np.random.uniform(low=posMin[i],high=posMax[i],size=dimensions))
#==== Velocity
for b in range(boidQuantity):#                                         init random velocities 
    #Convert to Unit Vector and scale according to random withine max
    l = math.sqrt(sum([math.pow(vel[b][i],2) for i in range(dimensions)]))
    r = maxVelMag*random.random()
    for i in range(dimensions):
        vel[b][i] = (vel[b][i]/l)*r 
#==== Acceleration
for b in range(boidQuantity):#                                         init random accelerations 
    #Convert to Unit Vector and scale according to random withine max
    l = math.sqrt(sum([math.pow(accel[b][i],2) for i in range(dimensions)]))
    r = maxVelMag*random.random()
    for i in range(dimensions):
        accel[b][i] = (accel[b][i]/l)*r
for _ in range(0,boidQuantity):#                                         init positions
    pos.append(np.array([random.randint(posMin[i],posMax[i]) for i in range(0,dimensions)]))

### (Optional) Manual Check of init DATA

In [6]:
q = 5
print(str(pos[0:min(len(pos),q)]))
print("================================================================================================================")
print(str(vel[0:min(len(vel),q)]))
print("================================================================================================================")
print(str(accel[0:min(len(accel),q)]))

[array([249.28592494,  30.25718274, 172.2136173 ]), array([548.47197631, 461.46417428, 285.24963683]), array([303.14772616, 566.71863436, 640.84169656]), array([ 47, 536, 243]), array([142, 572, 589])]
[array([0.64290556, 0.98357674, 0.96883067]), array([1.18468322, 0.93456605, 1.58406593]), array([0.83544712, 0.84346802, 0.86044584]), array([0.20040681, 0.91359243, 0.00255579]), array([1.52093737, 1.02047718, 0.73444776])]
[array([0.09544766, 0.53867636, 0.47494293]), array([0.06605385, 0.08716428, 0.05014399]), array([0.00060702, 0.35485826, 0.0081503 ]), array([2.41219863, 0.29641196, 1.13891762]), array([1.48252686, 0.97362231, 0.64239801])]


### Create objects for: Initial Data, Settings, Configurations

In [7]:
#Make it easier to pass all the data to the simulation class and access it withine that class
initdata = {# intial data as well as storage for the calculation (will be modified during the calculation)
    "position":pos,
    "velocity":vel,
    "acceleration":accel,#technically not needed.... more for observational purposes... if desired
    "mass":mass
}
settings={# the settings for running the simulation
    "boidQuantity":boidQuantity,
    "minPositions":posMin,
    "maxPositions":posMax,
    "maxForceMag":maxForceMag,
    "maxAccelerationMag":maxAccelMag,
    "maxVelocityMag":maxVelMag,
    "boidScale":boidScale,
    "active":active,
    "boidColor":boidColor,
    "boidColorRGBList":boidColorRGBList
}
configurations={# the configured rules for the interactions...
    "ruleList":ruleList,
    "ruleDistances":ruleDistances,
    "ruleWeights":ruleWeights
}

### Boid Utilities (Old Code Modified)

In [8]:
class BoidUtils:
    def __init__(self):
        pass
    @staticmethod
    def calcMag(npArray):
        return np.linalg.norm(npArray)
        #return math.sqrt(sum([math.pow(npArray[i],2) for i in range(npArray.size())]))
    @staticmethod
    def limitMag(npArray,maxMag):
        mag = BoidUtils.calcMag(npArray)
        if mag>maxMag:
            return npArray*(maxMag/mag)
        else:
            return npArray
    @staticmethod
    def calc_neighbors(sub_map, limiting_distance = None):
        if limiting_distance == None:#if a limiting distance is not specified the rule does not use it so exit without extra 
            #computation.... 
            return None
        #given an array of connecting weights returns the keys (indicies) of items that fall within the limiting dist
        neighbor_indx = []
        for key in sub_map:
            if sub_map[key] < limiting_distance:
                neighbor_indx.append(int(key))
        return neighbor_indx
            
    @staticmethod
    def timeit(method):
        #I TAKE NO CREDIT FOR THIS timeit CODE! all credit to the author at the link below...
        # CREDIT: https://medium.com/pythonhive/python-decorator-to-measure-the-execution-time-of-methods-fa04cb6bb36d
        def timed(*args, **kw):
            ts = time.time()
            result = method(*args, **kw)
            te = time.time()
            if 'log_time' in kw:
                name = kw.get('log_name', method.__name__.upper())
                kw['log_time'][name] = int((te - ts) * 1000)
            else:
                print('%r  %2.4f ms' % (method.__name__, (te - ts) * 1000))
            return result
        return timed
    @staticmethod
    def calc_distance_mapV2(list_of_positions):
        #Using timeit on this determined that this was the slowest part of thecode ... V2 is the realization that
        #the first distance map while efficient ... was still technically O(n^2) not the more desired O([1/2]n^2 + [1/2]n)
        #this code fixes that...
        '''
        :description: uses a caching based approach to calculate a dictionaries of dictionaries containing the position distances:
                          A   to    B         dist
                      {index:{ otherindex: distance,....},....}
        :param list_of_positions: a list of position Vectors [Vector,Vector,...,Vector]
        :modular_space: see euclidian_distance docstring (note this may slow down by up to a factor of 4!!!
        squared: see euclidian_distance docstring
        :return: (see description)
        '''
        cache={}# really only gets a slim asyntotic advantage...
        for i in range(0,len(list_of_positions)):
            #Determine if the current entry is empty if so put a new array else preserve it
            if i not in cache:# at a certian point i takes on the value that j was ... so to prevent overriding
                cache[i] = {}
            #now we need to go through all the positions
            for j in range(len(list_of_positions)-1,i,-1):
                print(len(list_of_positions))
                dist = BoidUtils.calcMag(np.subtract(list_of_positions[i], list_of_positions[j]))
                #Now we add it to the two locations it will be A-B and B-A    --> cache[i][j] and cache[j][i]
                cache[i][j] = dist# we know cache[i] exists so add it to that dictionary
                #we are however not sure if cache[j] exists yet and if it does not we need to init it first (rather if there is a dictionary at key j)
                if j not in cache:#we need to have a dictionary there to put a value... make sure not to overwrite it...
                    cache[j]={}
                #now we can add the distance...
                cache[j][i] = dist
        return cache
    #=== Predefined Update rules! these may be used withine the BoidSwarm ... custom rules can be added...
    @staticmethod
    def CohereForce(boidIdx,dataObject,calculatedNeighborsIdxs,sett,conf,ruleIdx):# ALL RULES IMPLEMENT THIS SAME ARGs SIGNATURE
        '''
        :description: calculates the limited weighted coherernce force for a given boid 
        :param boidIdx: the index of the current boid for which the calculation is being done
        :param dataObject: a dictionary containing: position:[[x,y,z]...] velocity:[[x,y,z]...] ... etc... acceleration mass
        :param calculatedNeighborsIdx: a list of indicies for the position list containing the neighboring positions 
        :param sett:  the setting object reference
        :param conf: the config object reference
        :param ruleIdx: the index of the configuration weights for this rule...
        :return: the force to be applied limited by etc...
        '''        
        # store
        pos_sum = np.zeros(len(dataObject["position"][0]))# i.e if it is xyz ==> 3 xy==> the dimension of the vector...
        count = len(calculatedNeighborsIdxs)
        #if no neighbors then break returning zeros...
        if count==0:
            return pos_sum
        #iterate over the neighbors indicies
        for nIndex in calculatedNeighborsIdxs:
            #Calculate the center position weighted by mass
            pos_sum= np.add(pos_sum,dataObject["position"][nIndex]*dataObject["mass"][nIndex]) 
        #Calculate the target and return the limited weighted force
        target = pos_sum * (1/count)
        desired = BoidUtils.limitMag(np.subtract(target,dataObject["position"][boidIdx]),sett["maxVelocityMag"])
        steer = np.subtract(desired,dataObject["velocity"][boidIdx])
        return conf["ruleWeights"][ruleIdx]*BoidUtils.limitMag(steer,sett["maxForceMag"])#note rule weight always on [0,1]
            
    @staticmethod
    def SeparateForce(boidIdx,dataObject,calculatedNeighborsIdxs,sett,conf,ruleIdx):
        # store
        pos_sum = np.zeros(len(dataObject["position"][0]))# i.e if it is xyz ==> 3 xy==> the dimension of the vector...
        count = len(calculatedNeighborsIdxs)
        #if no neighbors then break returning zeros...
        if count==0:
            return pos_sum
        #iterate over the neighbors indicies
        for nIndex in calculatedNeighborsIdxs:
            #Calculate the center position weighted by mass
            diff = np.subtract(dataObject["position"][boidIdx],dataObject["position"][nIndex])
            edgeDist = BoidUtils.calcMag(diff)# the distance between the points
            edgeDist = math.pow(edgeDist,2)
            if edgeDist != 0:#Ignore ones whose edge distance is zero ... (also prevents div by zero)
                pos_sum= np.add(pos_sum,diff*(1/(edgeDist)))# equivilant to taking the norm of diff then weighing it by 1/edgedist
        #Calculate the target and return the limited weighted force
        target = pos_sum * (1/count)
        if not np.all(target):#if all zeros exit early ... mag will be 0 div by zero error to happen...
            return target
        steer = np.subtract((sett["maxVelocityMag"]/BoidUtils.calcMag(target))*target,dataObject["velocity"][boidIdx])
        
        return conf["ruleWeights"][ruleIdx]*BoidUtils.limitMag(steer,sett["maxForceMag"])#note rule weight always on [0,1]    
    
    @staticmethod
    def  AlignForce(boidIdx,dataObject,calculatedNeighborsIdxs,sett,conf,ruleIdx):
        # store
        vel_sum = np.zeros(len(dataObject["velocity"][0]))# i.e if it is xyz ==> 3 xy==> the dimension of the vector...
        count = len(calculatedNeighborsIdxs)
        #if no neighbors then break returning zeros...
        if count==0:
            return vel_sum
        #iterate over the neighbors indicies
        for nIndex in calculatedNeighborsIdxs:
            vel_sum= np.add(vel_sum,dataObject["velocity"][nIndex]) 
        #Calculate the target and return the limited weighted force
        target = vel_sum * (1/count) 
        steer = np.add((sett["maxVelocityMag"]/BoidUtils.calcMag(target))*target,dataObject["velocity"][boidIdx])        
        return conf["ruleWeights"][ruleIdx]*BoidUtils.limitMag(steer,sett["maxForceMag"])#note rule weight always on [0,1]
    @staticmethod
    def WallForce(boidIdx,dataObject,sett,conf):
        # if a boid goes out of bounds for a certain dimension add a force in the opposite direction...  that is
        # polynimially proportional to the distance==> max force seams like a good cap...
        force = np.zeros(len(dataObject["velocity"][0]))
        #for each component of the boids position make a force that increases polynomially with the distance past the bounds...
        for dim in range(force.size):#for each component
            if dataObject["position"][boidIdx][dim] < sett["minPositions"][dim]: 
                force[dim] += math.pow(sett["minPositions"][dim] - dataObject["position"][boidIdx][dim],2)
            if dataObject["position"][boidIdx][dim] > sett["maxPositions"][dim]: 
                force[dim] += math.pow(dataObject["position"][boidIdx][dim]-sett["maxPositions"][dim],2)
        #NOTE WALLFORCE IS NOT LIMITED ... We create an opposing force that grows with the squared distance in each dim
        return force
        

### Rule Definitions & Config

In [9]:
#================ Setup The Boid Rules...
configurations["ruleList"].append(BoidUtils.CohereForce)
configurations["ruleList"].append(BoidUtils.AlignForce)
configurations["ruleList"].append(BoidUtils.SeparateForce)
assert(len(configurations["ruleList"])==len(configurations["ruleWeights"])==len(configurations["ruleDistances"]))#err check

#Special force for bounding the simulation BoidUtils.WallForce is not setup as a normal force because it does not need neighbors
#to be calculated

### Boid Simulation Class

In [10]:
class BoidSimulation():
    def __init__(self,dat,sett,conf,recordPositionHistory=False):
        self.dat = dat
        self.sett = sett
        self.conf = conf
        self.rph = recordPositionHistory
        self.ph = []#position history ... updated after each timestep if applicable (assumes no changes to active durring game)

    def time_step(self):#Runs a timestep of the simulation... 
        #=== Calculate the next timestep for the swarm
        
        #TODO
        # build KD tree O(nlogn)
        # Query tree for each boid(n boids) for each rule (k rules) ==> O(nklogn) < O(n^2)  
        #                                   (note k is a constant and for most k where k is small)
        # Calculate the forces for all of the points returned by the querying criteria O(cnk) (where c = the average # of points returned for each...)
        # Bound the positions,velocities,accelerations O(n) 
        tree = KDTree(self.dat["position"])
        #Calculate the neighbors for each ruleset for each boid all in parallel?
        
        #For now implement calculating the distances using an optimized sequential algorithm
        #distance_map = BoidUtils.calc_distance_mapV2(self.dat["position"])
        
        
        #Calculate the position updates in parallel...?
        for i in range(0,self.sett["boidQuantity"]):# n boids
            #Only calculate for the active boids
            if self.sett["active"][i]==1:#if the boid is active... then we want to run the calculation...
                #Calculate the neighbors for a certain rule
                forceSumForBoid= np.zeros(len(self.dat["acceleration"][0]))# just use one of the variables to get the desired dim
                for r in range(0,len(self.conf["ruleList"])):# k rules....
                    #for each rule compute the neighbors ... find the forces update etc..
                    neighborIndxs = tree.query_ball_point(self.dat["position"][i],self.conf["ruleDistances"][r])# O(logn)
                    #neighborIndxs = BoidUtils.calc_neighbors(distance_map[i],self.conf["ruleDistances"][r])# THIS IS A VERY COSTLY FUNCTION...
                    #Calculate the force update by getting the function for the update... (rules limit force... & soft bound)
                    update = self.conf["ruleList"][r](i,self.dat,neighborIndxs,self.sett,self.conf,r) 
                    for i in range(update.size):
                        if np.isnan(update[i]):
                            print(r)
                            raise Exception()
                    #Add the force
                    forceSumForBoid = np.add(forceSumForBoid,update)
                #Now apply the force to the boid ... factoring in the boids mass first anad adding in the special wall force
                accelUpdate = (forceSumForBoid+BoidUtils.WallForce(i,self.dat,self.sett,self.conf))  * (1/self.dat["mass"][i])
                #Update the acceleration velocity and position
                self.dat["acceleration"][i] = accelUpdate
                self.dat["velocity"][i] = BoidUtils.limitMag(np.add(self.dat["velocity"][i],self.dat["acceleration"][i]),self.sett["maxVelocityMag"])
                self.dat["position"][i] = np.add(self.dat["position"][i],self.dat["velocity"][i])
        #Should we save the data?
        if self.rph:
            self.ph.append(copy.deepcopy(self.dat["position"]))
        #DONE

### Controls 

In [11]:
#=============== Controls Setup
#TODO: slider for each rule distance & weight
#      scatter button
#      3D snapshot ==> pause simulation and use pptk to Draw the 3d frame of the simulation...

#well the last hour was spent trying to get sliders to work ... the issue is that the functions can only recieve the 
#updated value and rerun however the function does not update the value of an outside variable... these are not at 
#all similar to 'event based' despite it being called such... 
# FOR NOW: this feature of the code will be put on halt

### Screen Setup & Definitions for Functions

In [16]:
#================ Screen Defs for 2D
class Graph2D():
    def __init__(self):
        self.graph = None
        pass
    def createGraph(self,posMax,posMin,backgroundColor):#Create a tkinter canvas to draw to...
        #Creates a tkinter canvas with the specified properties:
        #posMin: a vector (as a list) contiaining the minimum bounds for the screen in the x,y,z ... k for any n dimensional vector
        #posMax: a vector (as a list) contiaining the minimum bounds for the screen in the x,y,z ... k for any n dimensional vector
        #backgroundColor: the background color of the canvas specified by color name string 'white' or by rgb array (255,255,255)
        root = Tk()
        _width,_height = (posMax[0]-posMin[0]),(posMax[1]-posMax[1])
        root.geometry('%dx%d+%d+%d' % (_width, _height, (root.winfo_screenwidth() - _width) / 2, (root.winfo_screenheight() - _height) / 2))
        root.bind_all('<Escape>', lambda event: event.widget.quit())
        graph = Canvas(root, width=_width, height=_height, background=backgroundColor)
        graph.pack()
        self.graph= graph
    def drawBoids(self,posList,activeList,boidScale,boidColor):#Draw the Boids
        #this function takes a tinketer canvas (graph) and looks at all the positions posList and if the 
        #item is active  (specified by ) activeList 1=active 0=not then it draws the boid as a circle at that
        #position with the scale and color specified.. scale and color is shared among all the boids...
        #(NOTE) depending on how the code for the simulation utilizes the CPU it might be beneficial to 
        #run the graph drawing in a separate thread then syncronize before the next loop ==> i.e let the next calculation
        #run while the previous is being drawn or something in this manner
        #=== Draw the swarm
        self.graph.delete(ALL)#                  Clear the previous screen
        for p in posList:#for every position ==> O(n)
            if activeList[i]==1:#if the boid is active...
                #draw a circle whose center is at the current position of the boid and with a radius of the scale factor..
                d2Cord = (p[0] + boidScale,#lower left x     the coordinates for the bounding corners of the circle
                          p[1] +boidScale,#lower left y      this keeps the boid in the center
                          p[0] - boidScale,#upper right x
                          p[1] - boidScale)#upper right y
                #draw the boid ... note using syntatic sugar for position update
                self.graph.create_oval(d2Cord,fill=boidColor)
        self.graph.update()#Update the screen...
    

#==============Screen Defs 3D
class Graph3D():#(Limitation: cant hide non active boids ... without doing an extra O(n) filter )
    def __init__(self,boidColor,boidScale):
        #Save the color
        self.boidColor = boidColor #passed rgb list
        #Setup
        self.pcd = open3d.geometry.PointCloud()
        self.vis = open3d.visualization.Visualizer()
        #Create the Window
        self.vis.create_window()
        self.vis.add_geometry(self.pcd)
        self.isInitialFrame = True # The frist frame determines the 
        #Determine Hoow big the points are
        render_option = self.vis.get_render_option()
        render_option.point_size = boidScale
    def drawBoids(self,posList,timeDelay=0.05):
        #setup the frame
        frame = open3d.Vector3dVector(posList)
        #Set the points (frame) to be drawn
        self.pcd.points = frame
        #Is it the first frame? if so set colors
        if self.isInitialFrame:      
            #Set the color based upon the first Number of boids
            self.pcd.colors =  open3d.utility.Vector3dVector([self.boidColor for i in range(0,len(posList))])
            self.isInitialFrame = False
        #Check to make sure the length of the position list is the same as the 
        '''
        assert(not (len(pcd.colors)==len(posList)),
               "The colors for the points are set on the first \
                frame and not recomputed each time as that would be slightly computationally expensive...\
                so the number of positions cannot change between runs unless that is coded")'''
        #Draw it to the screen
        to_reset_view_point = True 
        #Update the geometry
        self.vis.update_geometry()
        if to_reset_view_point:
            self.vis.reset_view_point(True)
            to_reset_view_point = False
        self.vis.poll_events()
        self.vis.update_renderer()
        time.sleep(timeDelay)
        
    def closeWindow():
        self.vis.destroy_window()

### Simulation Loop

In [17]:
#================ Simulation Loop
#=== Setup
#2D
#graphHolder2D = Graph2D()
#graphHolder2D.createGraph(posMax,posMin,backgroundColor)# 2D Draw (actually slower than 3D bc it draws sequentially)
#3D (graph auto created ...)
#graphHolder3D = Graph3D(boidColorRGBList,boidScale)
graphHolder3D = Graph3D([0,0,0],5)
#Simulation
sim = BoidSimulation(initdata,settings,configurations)
#=== Simulation Loop
maxFrameCtr=50
while maxFrameCtr>0:
    maxFrameCtr-=1
    #=============================== COMPUTE
    #Use to see how much time has passed between frames so to limit the frame rate when computed too fast... 
    #(a nice problem to have)
    startTime = time.time()
    #============================================================================
    sim.time_step()
    #============================================================================
    #=== Draw the swarm & Update the screen
    #2D
    #graphHolder2D.drawBoids(sim.dat["position"],
    #                        sim.sett["active"],
    #                        sim.sett["boidScale"],
    #                       sim.sett["boidColor"])
    #3D
    graphHolder3D.drawBoids(sim.dat["position"])
    time.sleep(0.5)
    '''
    q = 5
    print(str(sim.dat["position"][0:min(len(sim.dat["position"]),q)]))
    print("================================================================================================================")
    print(str(sim.dat["velocity"][0:min(len(sim.dat["velocity"]),q)]))
    print("================================================================================================================")
    print(str(sim.dat["acceleration"][0:min(len(sim.dat["acceleration"]),q)]))
    print("----------------------------------------------------------------------------------------------------------------")
    time.sleep(2)
    
    
    #=== Prevents frames from being drawn too fast: 
    endTime = time.time()
    timeDelta = endTime-startTime
    if timeDelta < minFrameTime:
        time.sleep(minFrameTime-timeDelta)
    '''

IndexError: index 100 is out of bounds for axis 0 with size 100

In [ ]:
 def update_position(self,boids_list,distance_map,canvas_bounds):
        '''
        :description:  uses the velocity and the modular space constraints (screen height and width) to compute the next
                       position of the boid; NOTE! Velocity should manually be updated FIRST!
        :param distance_map: a dictionary containing the distances between this boid and all other boids
                             key: index value: distance
        :param canvas_bounds: is a Vector specifying the maximum value for x and y coordinates ex Vector([1920,1080])
        '''

        #Calculate the weight updates according to the boid rules...

        # F=MA ==> A = F/M  ==> Force F applied to Mass M induce Acceleration A
        #Determine the forces of all the other boids acting upon
        # fc = (sigma all forces)/m
        # accel |--> newaccel = fc.norm()*max_accel if abs(fc)>max accel for either component else fc
        # vel   |--> newvel = (vel+accel).norm()*max_vel if abs(vel+accel)>max_vel else  vel+accel
        # pos   |--> newpos = (pos + vel) mod (canvas bounds)      make sure to convert to ints ... for game grid
        # accel |--> 0
        #The Sum of the Forces from the Rules
        force_sum = Vector.zeros(dim=self.pos.shape())
        for i in range(0,len(self.rules)):
            force_sum = force_sum + self.rules[i](self,boids_list,distance_map,self.rule_weights[i],self.rule_ranges[i])
        force_sum = force_sum*(1/self.mass)#factor in the mass of the boid...
        #Update the Acceleration according to the force and limit: (if either component is over)
        self.accel = force_sum.limit(self.max_accel)
        #Update the Velocity and Limit
        vel_update = self.vel+self.accel
        self.vel = vel_update.limit(self.max_vel)
        #Update the position and wrap the bounds...   add the current position and velocity then convert to int then mod canvas bounds
        self.pos = ((self.pos+self.vel).apply(int)) % canvas_bounds
        #finally reset the acceleration
        self.accel=Vector.zeros(dim=canvas_bounds.shape())